# Reddit Company Sentiment Analysis

Use a LangChain agent with the `social_media_search` tool to research how a company is being discussed on Reddit. The agent autonomously searches Reddit for discussions, opinions, and sentiment about any company.

**What you'll learn:**
- Using `social_media_search` from `tavily_agent_toolkit` with `platform="reddit"`
- Building a LangChain agent for sentiment analysis
- Streaming agent tool calls for visibility into the research process

## Setup

In [1]:
%pip install -qU tavily-python langchain-openai langgraph python-dotenv tavily-agent-toolkit


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [13]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY:\n")

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY:\n")

## Define the Reddit Search Tool

Wrap the `social_media_search` tool from `tavily_agent_toolkit` with `platform="reddit"` for Reddit-specific searches.

In [14]:
import os
from typing import Literal, Optional
from langchain_core.tools import tool
from tavily_agent_toolkit import social_media_search

TAVILY_API_KEY = os.environ.get("TAVILY_API_KEY", "")

@tool
def search_reddit(
    query: str,
    max_results: int = 5,
    time_range: Optional[Literal["day", "week", "month", "year"]] = "month",
) -> dict:
    """
    Search Reddit for discussions, opinions, and threads.
    
    Args:
        query: Search query - include company name and relevant keywords
        max_results: Number of results to return (default: 5)
        time_range: Filter by time - "day", "week", "month", or "year"
    
    Returns:
        Dictionary with Reddit search results including post titles, URLs, and content.
    """
    return social_media_search(
        query=query,
        api_key=TAVILY_API_KEY,
        platform="reddit",
        include_raw_content=True,
        max_results=max_results,
        include_answer=True,
        time_range=time_range,
    )

## Create the Research Agent

Build a LangChain agent that uses the Reddit search tool to analyze company sentiment.

In [15]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent

SYSTEM_PROMPT = """You are a social media research analyst specializing in Reddit sentiment analysis.

Your task is to research how a company is being discussed on Reddit. Search multiple times with different query variations to get a complete picture:
- Company name + "review" or "experience"
- Company name + "problems" or "issues"
- Company name + "vs" competitors
- Company name + product/service names

Analyze the discussions and provide:
1. **Overall Sentiment**: Positive, negative, or mixed
2. **Key Themes**: What topics come up most often
3. **Praise**: What people like about the company
4. **Criticism**: Common complaints or concerns
5. **Notable Quotes**: Interesting comments that capture the sentiment

Include inline citations [1], [2], [3]... and list all sources with URLs at the end."""

model = ChatOpenAI(model="gpt-5-mini")

agent = create_agent(
    model=model,
    tools=[search_reddit],
    system_prompt=SYSTEM_PROMPT,
)

## Stream Agent Execution

Helper function to stream agent responses and show tool calls in real-time.

In [16]:
async def stream_research(agent, query: str, max_tool_calls: int = 5) -> str:
    """Stream agent execution, printing tool calls, and return final response.
    
    Args:
        agent: The LangChain agent to run
        query: The user's query
        max_tool_calls: Maximum number of tool call iterations (default: 5)
    """
    final_response = None
    inputs = {"messages": [{"role": "user", "content": query}]}
    config = {"recursion_limit": max_tool_calls}
    
    async for chunk in agent.astream(inputs, stream_mode="updates", config=config):
        for node_output in chunk.values():
            for msg in node_output.get("messages", []):
                if hasattr(msg, "tool_calls") and msg.tool_calls:
                    for tool_call in msg.tool_calls:
                        query_arg = tool_call.get("args", {}).get("query", "")
                        print(f" Searching Reddit: \"{query_arg}\"")
                elif hasattr(msg, "name") and msg.name:
                    print(f" Search completed")
                elif hasattr(msg, "content") and msg.content:
                    final_response = msg.content
    
    return final_response or "No response generated"

## Run the Analysis

Research how a company is being discussed on Reddit. Try different companies to see varied results.

In [ ]:
from IPython.display import display, Markdown

# Change this to any company you want to research
COMPANY = "Tesla"

print(f"Researching Reddit sentiment for: {COMPANY}")
print("=" * 50)

report = await stream_research(
    agent, 
    f"How is {COMPANY} being talked about on Reddit? What do people think of them?"
)

print("\n" + "=" * 50)
print("ANALYSIS COMPLETE")
print("=" * 50 + "\n")

display(Markdown(report))

Researching Reddit sentiment for: Tesla
 Searching Reddit: "Tesla review experience reddit"
 Searching Reddit: "Tesla problems issues reddit"
 Searching Reddit: "Tesla vs Rivian reddit"
 Searching Reddit: "Tesla vs Ford reddit"
 Searching Reddit: "Model 3 review reddit"
 Searching Reddit: "Autopilot problems reddit"
 Searching Reddit: "FSD review reddit"
 Searching Reddit: "Tesla service supercharger reddit"
 Search completed
 Search completed
 Search completed
 Search completed
 Search completed
 Search completed
 Search completed
 Search completed

ANALYSIS COMPLETE



Summary (short): Overall sentiment on Reddit is mixed. Many owners rave about driving experience, range, and Tesla’s software/FSD, but there are frequent, persistent complaints about build quality, reliability, and service/support. Below I break down the main themes, what people praise, what they criticize, and include representative quotes and sources.

1) Overall sentiment
- Mixed: positive owner enthusiasm for the product and software features, combined with recurring negative reports about reliability, build quality, and service experience [1][2][4][5][6][10].

2) Key themes (what shows up most)
- Driving experience: acceleration, quiet ride, one‑pedal driving praised frequently [2][3][12].
- Full Self Driving / Autopilot: strong interest and polarized reports — many praise highway performance, others call it inconsistent or risky in complex/city scenarios [10][11][12].
- Build quality & reliability: repeated reports of fit/finish issues and component failures, especially on used cars and early delivery windows [5][6].
- Service experience & wait times: long waits for service appointments and slow fixes are common complaints [6][13].
- Charging / Supercharger network: often praised for road trips (but occasional availability/queue complaints appear in some threads) [11].
- Comparisons vs competitors: Tesla usually wins on software and charging/efficiency; competitors (e.g., Rivian) are praised for interior fit and finish and perceived build quality [8][9].

3) Praise — what people like
- Driving dynamics: smooth, quiet, fast acceleration and enjoyable driving feel (Model 3/Model Y owners often report this) [2][3][12].
- Software & navigation: over‑the‑air updates, navigation, and software features (including FSD for some users) are frequently called out positively [11][12].
- Charging ecosystem / road‑trip practicality: Supercharger network and long‑range capability make long trips practical for many owners [11].
- Minimalist interior and usable cabin space: many owners like the interior simplicity and practicality [3][2].

Representative praise citation: “Car is amazing. Love how smooth and quiet the ride is… 0–60 in 3 seconds and it's quiet as a mouse!” [2].

4) Criticism — common complaints and concerns
- Build quality / fit & finish: dents, gaps, paint, panel alignment, and other manufacturing defects remain common complaints, especially for new deliveries and some used cars [5][6].
- Reliability and component issues: recurring service visits for some vehicles; some threads claim high rates of early service visits for new owners [6][5].
- Service center experience: long waits for appointments, multi‑week repair timelines, and mixed warranty support experiences [6][13].
- Inconsistent software/feature behavior: Autopilot/FSD can be excellent on highways but unreliable in complex environments; odd software glitches and UX annoyances (e.g., charge port unlocking, speed-limiting behavior) show up regularly [4][10][2].
- Customer support / corporate decisions: criticism of how Tesla handles certain recalls, communications, or policy changes appears in broader threads [5].

Representative criticism citations: “Tesla is the most unreliable used car brand in America…” [5]; “Reports suggest up to 70% of new Tesla owners need a service visit within 30 days” [6]; examples of software annoyances like charge port not unlocking and unexpected speed limits when using FSD [4].

5) Comparisons with competitors
- Tesla = software, charging network, range, and driving feel.
- Rivian/other legacy OEMs = often better interior fit & finish and (per some posts) more traditional dealer/service support; reviewers advise weighing software advantages vs. build quality/service tradeoffs [8][9].

6) Notable quotes (representative / capture sentiment)
- Positive owner: “I have a 2025 Model 3, and overall I love this car. It is the best car I’ve ever had by a lot.” — Model 3 owner listing small annoyances but overall high satisfaction [4].
- FSD nuance: “FSD is awesome on highways, handling lane changes and cruising smoothly. It's a great foundation, but city driving needs some guardrails.” [10].
- Reliability concern: “Tesla is the most unreliable used car brand in America, …” (thread discussing used‑car reliability stats and anecdotes) [5].
- Competitor praise: “Rivian has a significantly better interior — Tesla is like riding in an eggshell.” (user comparing fit/finish) [8].
- Service worry: “It’s overall been a good car but twice it’s had problems and Tesla has been astonishingly slow to fix them. They took 14 weeks to fix one motor issue.” — owner experience in Europe [13].

7) Quick takeaways for different audiences
- Potential buyers who prioritize driving feel, range, software features, or the Supercharger network: many owners are extremely happy — but budget time/expectations for possible service visits and minor defects [2][11][12].
- Buyers who prioritize perfect fit & finish, quick dealer service, or conservative reliability records: some competitors may be a better fit (Rivian, legacy OEM EV offerings) [8][9].
- EV enthusiasts and FSD-interested buyers: Reddit contains both evangelists and skeptics; FSD is frequently called impressive on highways but still not universally trusted for all city/complex situations [10][11][12].

Sources (representative Reddit threads cited above)
1. https://www.reddit.com/r/TeslaLounge/comments/1lmvu0l/for_anyone_considering_a_tesla_my_1_weekish_review/
2. https://www.reddit.com/r/TeslaModel3/comments/1kyf2yy/one_month_review/
3. https://www.reddit.com/r/TeslaModelY/comments/1mhdg4n/new_model_y_owner_my_review_so_far/
4. https://www.reddit.com/r/teslamotors/comments/1lu9n7x/model_3_feedback/
5. https://www.reddit.com/r/technology/comments/1pj1rri/tesla_is_the_most_unreliable_used_car_brand_in/
6. https://www.reddit.com/r/TeslaModelY/comments/1ka1ech/model_y_issues/
7. https://www.reddit.com/r/RealTesla/comments/1ja96gb/serious_question_about_teslas_former_build_quality/
8. https://www.reddit.com/r/Rivian/comments/1kvrabf/rivian_vs_tesla_thoughts/
9. https://www.reddit.com/r/Rivian/comments/1im7xav/rivian_vs_tesla_probably_not_the_right_comparison/
10. https://www.reddit.com/r/TeslaFSD/comments/1jczru5/1000mile_review_of_fsd/
11. https://www.reddit.com/r/TeslaFSD/comments/1of8zl0/daily_fsd_driver_1413_review_model_y_hw4/
12. https://www.reddit.com/r/TeslaModel3/comments/1ib0w4i/fsd_is_really_good/
13. https://www.reddit.com/r/evs_ireland/comments/1i53shi/owners_experience_with_a_model_3/

If you want, I can:
- Expand this with sentiment scoring (percent positive/negative) across a larger sample of Reddit posts,
- Provide a timeline (how sentiment has trended recently),
- Or focus on one subreddit (e.g., r/TeslaModel3, r/TeslaFSD, r/RealTesla) for deeper analysis. Which would you prefer?

## Try Different Companies

Run additional analyses on different companies to compare sentiment.

In [ ]:
# Uncomment and run to analyze another company

# COMPANY = "OpenAI"
# report = await stream_research(
#     agent, 
#     f"How is {COMPANY} being talked about on Reddit? What do people think of them?"
# )
# display(Markdown(report))

## Next Steps

- Expand to multiple platforms by changing `platform="reddit"` to `platform="combined"` or other platforms
- Add time-based tracking to monitor sentiment changes over time
- See the [Social Media Research Agent](../../use-cases/langgraph/social_media_research.py) for a full CLI implementation
- Explore the [social_media_search source](../../agent-toolkit/tools/social_media.py) to understand the underlying implementation